In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

First of all we are going to check out the train.csv which, according to the input data description:

includes the target unit_sales by date, store_nbr, and item_nbr and a unique id to label rows.

The target unit_sales can be integer (e.g., a bag of chips) or float (e.g., 1.5 kg of cheese).
Negative values of unit_sales represent returns of that particular item.

The onpromotion column tells whether that item_nbr was on promotion for a specified date and store_nbr.
Approximately 16% of the onpromotion values in this file are NaN.

NOTE: The training data does not include rows for items that had zero unit_sales for a store/date combination. There is no information as to whether or not the item was in stock for the store on the date, and teams will need to decide the best way to handle that situation. Also, there are a small number of items seen in the training data that aren't seen in the test data.

In [2]:
train = pd.read_csv('../input/train.csv')

In [3]:
train.head()

In [4]:
print("we are dealing with ",len(train), "rows")

which are TOOOO much ! so we are going to work with  sample

In [5]:
strain = train.sample(frac=0.01,replace=True)

## Items

In [6]:
items = pd.read_csv('../input/items.csv')

In [7]:
items.head()

In [8]:
print("we are dealing with",len(items['family'].unique()),"families of products")

In [9]:
f,axarr = plt.subplots(1,1,figsize=(15,10))
plt.xticks(rotation='vertical')
fhist = items.groupby(['family'],as_index=False).count()
sns.barplot(x=fhist['family'],y=fhist['class'])


## Stores

In [10]:
stores = pd.read_csv('../input/stores.csv')
stores.head()

In [11]:
print("we are dealing with",len(stores['city'].unique()),'cities and',len(stores['state'].unique()),'states')

In [12]:
f,axar = plt.subplots(1,1,figsize=(15,10))
plt.xticks(rotation='vertical')
shist = stores.groupby(['city'],as_index=False).count()
sns.barplot(x=shist['city'],y=shist['state'])

## Transactions

In [13]:
trans = pd.read_csv('../input/transactions.csv')

In [14]:
trans.head()

## Join Data 

In [15]:
df = strain.merge(right = items, on='item_nbr')

In [16]:
df = df.merge(right=stores,on='store_nbr')

In [17]:
df.head()

Now I am interested to see how things change in time or if there are any possible recurring patterns. So I m going to turn dates from strings to datetime objects which enables for same interesting query like day of th year, day of the week, etc.

In [18]:
df['date']=pd.to_datetime(df['date'])

In [19]:
df['yday']=df['date'].dt.dayofyear

In [22]:
df['year']=df['date'].dt.year

In [24]:
df['month']=df['date'].dt.month

In [20]:
stores=[df['store_nbr'].unique()]

In [26]:
df.head()

Let's create groups for store, year, month and product family.

In [30]:
grouped = df.groupby(['store_nbr','year','month','family']).sum().drop(['id','item_nbr','class','perishable','cluster','yday'],axis=1)

In [34]:
grouped.head(20)

As you can see we now have a table in which for each store we know how many products of each family have been sold ordered by years and months. 